# 01 - Data Ingestion: SEC 10-K Filings

This notebook handles downloading and organizing SEC 10-K filings for S&P 100 companies.

**Objectives:**
- Download 10-K filings (HTML/PDF) from SEC EDGAR
- Download XBRL data for cross-validation
- Organize files by company and fiscal year
- Target: 50-150 filings (3-5 years per company)

In [14]:
import os
import sys
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime
import time
from tqdm import tqdm

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))

from utils.config import DATA_DIR, RAW_DATA_DIR
from utils.sec_api import SECDownloader

## 1. Setup and Configuration

In [15]:
# SEC EDGAR API Configuration
# IMPORTANT: Replace with your email for SEC API compliance
USER_AGENT = "YourName your.email@example.com"

# Target parameters
TARGET_YEARS = 3  # Number of fiscal years per company
START_YEAR = 2020
END_YEAR = 2024

# File format preference
PREFER_HTML = True  # Prefer HTML over PDF for cleaner tables

## 2. Load S&P 100 Companies List

In [16]:
sp500_companies = [
    {"ticker": "MMM", "cik": "0000066740", "name": "3M"},
    {"ticker": "AOS", "cik": "0000091142", "name": "A. O. Smith"},
    {"ticker": "ABT", "cik": "0000001800", "name": "Abbott Laboratories"},
    {"ticker": "ABBV", "cik": "0001551152", "name": "AbbVie"},
    {"ticker": "ACN", "cik": "0001467373", "name": "Accenture"},
    {"ticker": "ADBE", "cik": "0000796343", "name": "Adobe Inc."},
    {"ticker": "AMD", "cik": "0000002488", "name": "Advanced Micro Devices"},
    {"ticker": "AES", "cik": "0000874761", "name": "AES Corporation"},
    {"ticker": "AFL", "cik": "0000004977", "name": "Aflac"},
    {"ticker": "A", "cik": "0001090872", "name": "Agilent Technologies"},
    {"ticker": "APD", "cik": "0000002969", "name": "Air Products"},
    {"ticker": "ABNB", "cik": "0001559720", "name": "Airbnb"},
    {"ticker": "AKAM", "cik": "0001086222", "name": "Akamai Technologies"},
    {"ticker": "ALB", "cik": "0000915913", "name": "Albemarle Corporation"},
    {"ticker": "ARE", "cik": "0001035443", "name": "Alexandria Real Estate Equities"},
    {"ticker": "ALGN", "cik": "0001097149", "name": "Align Technology"},
    {"ticker": "ALLE", "cik": "0001579241", "name": "Allegion"},
    {"ticker": "LNT", "cik": "0000352541", "name": "Alliant Energy"},
    {"ticker": "ALL", "cik": "0000899051", "name": "Allstate"},
    {"ticker": "GOOGL", "cik": "0001652044", "name": "Alphabet Inc. (Class A)"},
    {"ticker": "GOOG", "cik": "0001652044", "name": "Alphabet Inc. (Class C)"},
    {"ticker": "MO", "cik": "0000764180", "name": "Altria"},
    {"ticker": "AMZN", "cik": "0001018724", "name": "Amazon"},
    {"ticker": "AMCR", "cik": "0001748790", "name": "Amcor"},
    {"ticker": "AEE", "cik": "0001002910", "name": "Ameren"},
    {"ticker": "AEP", "cik": "0000004904", "name": "American Electric Power"},
    {"ticker": "AXP", "cik": "0000004962", "name": "American Express"},
    {"ticker": "AIG", "cik": "0000005272", "name": "American International Group"},
    {"ticker": "AMT", "cik": "0001053507", "name": "American Tower"},
    {"ticker": "AWK", "cik": "0001410636", "name": "American Water Works"},
    {"ticker": "AMP", "cik": "0000820027", "name": "Ameriprise Financial"},
    {"ticker": "AME", "cik": "0001037868", "name": "Ametek"},
    {"ticker": "AMGN", "cik": "0000318154", "name": "Amgen"},
    {"ticker": "APH", "cik": "0000820313", "name": "Amphenol"},
    {"ticker": "ADI", "cik": "0000006281", "name": "Analog Devices"},
    {"ticker": "AON", "cik": "0000315293", "name": "Aon plc"},
    {"ticker": "APA", "cik": "0001841666", "name": "APA Corporation"},
    {"ticker": "APO", "cik": "0001858681", "name": "Apollo Global Management"},
    {"ticker": "AAPL", "cik": "0000320193", "name": "Apple Inc."},
    {"ticker": "AMAT", "cik": "0000006951", "name": "Applied Materials"},
    {"ticker": "APTV", "cik": "0001521332", "name": "Aptiv"},
    {"ticker": "ACGL", "cik": "0000947484", "name": "Arch Capital Group"},
    {"ticker": "ADM", "cik": "0000007084", "name": "Archer Daniels Midland"},
    {"ticker": "ANET", "cik": "0001596532", "name": "Arista Networks"},
    {"ticker": "AJG", "cik": "0000354190", "name": "Arthur J. Gallagher & Co."},
    {"ticker": "AIZ", "cik": "0001267238", "name": "Assurant"},
    {"ticker": "T", "cik": "0000732717", "name": "AT&T"},
    {"ticker": "ATO", "cik": "0000731802", "name": "Atmos Energy"},
    {"ticker": "ADSK", "cik": "0000769397", "name": "Autodesk"},
    {"ticker": "ADP", "cik": "0000008670", "name": "Automatic Data Processing"},
    {"ticker": "AZO", "cik": "0000866787", "name": "AutoZone"},
    {"ticker": "AVB", "cik": "0000915912", "name": "AvalonBay Communities"},
    {"ticker": "AVY", "cik": "0000008818", "name": "Avery Dennison"},
    {"ticker": "AXON", "cik": "0001069183", "name": "Axon Enterprise"},
    {"ticker": "BKR", "cik": "0001701605", "name": "Baker Hughes"},
    {"ticker": "BALL", "cik": "0000009389", "name": "Ball Corporation"},
    {"ticker": "BAC", "cik": "0000070858", "name": "Bank of America"},
    {"ticker": "BAX", "cik": "0000010456", "name": "Baxter International"},
    {"ticker": "BDX", "cik": "0000010795", "name": "Becton Dickinson"},
    {"ticker": "BRK.B", "cik": "0001067983", "name": "Berkshire Hathaway"},
    {"ticker": "BBY", "cik": "0000764478", "name": "Best Buy"},
    {"ticker": "TECH", "cik": "0000842023", "name": "Bio-Techne"},
    {"ticker": "BIIB", "cik": "0000875045", "name": "Biogen"},
    {"ticker": "BLK", "cik": "0002012383", "name": "BlackRock"},
    {"ticker": "BX", "cik": "0001393818", "name": "Blackstone Inc."},
    {"ticker": "XYZ", "cik": "0001512673", "name": "Block, Inc."},
    {"ticker": "BK", "cik": "0001390777", "name": "BNY Mellon"},
    {"ticker": "BA", "cik": "0000012927", "name": "Boeing"},
    {"ticker": "BKNG", "cik": "0001075531", "name": "Booking Holdings"},
    {"ticker": "BSX", "cik": "0000885725", "name": "Boston Scientific"},
    {"ticker": "BMY", "cik": "0000014272", "name": "Bristol Myers Squibb"},
    {"ticker": "AVGO", "cik": "0001730168", "name": "Broadcom"},
    {"ticker": "BR", "cik": "0001383312", "name": "Broadridge Financial Solutions"},
    {"ticker": "BRO", "cik": "0000079282", "name": "Brown & Brown"},
    {"ticker": "BF.B", "cik": "0000014693", "name": "Brown–Forman"},
    {"ticker": "BLDR", "cik": "0001316835", "name": "Builders FirstSource"},
    {"ticker": "BG", "cik": "0001996862", "name": "Bunge Global"},
    {"ticker": "BXP", "cik": "0001037540", "name": "BXP, Inc."},
    {"ticker": "CHRW", "cik": "0001043277", "name": "C.H. Robinson"},
    {"ticker": "CDNS", "cik": "0000813672", "name": "Cadence Design Systems"},
    {"ticker": "CZR", "cik": "0001590895", "name": "Caesars Entertainment"},
    {"ticker": "CPT", "cik": "0000906345", "name": "Camden Property Trust"},
    {"ticker": "CPB", "cik": "0000016732", "name": "Campbell's Company (The)"},
    {"ticker": "COF", "cik": "0000927628", "name": "Capital One"},
    {"ticker": "CAH", "cik": "0000721371", "name": "Cardinal Health"},
    {"ticker": "KMX", "cik": "0001170010", "name": "CarMax"},
    {"ticker": "CCL", "cik": "0000815097", "name": "Carnival"},
    {"ticker": "CARR", "cik": "0001783180", "name": "Carrier Global"},
    {"ticker": "CAT", "cik": "0000018230", "name": "Caterpillar Inc."},
    {"ticker": "CBOE", "cik": "0001374310", "name": "Cboe Global Markets"},
    {"ticker": "CBRE", "cik": "0001138118", "name": "CBRE Group"},
    {"ticker": "CDW", "cik": "0001402057", "name": "CDW Corporation"},
    {"ticker": "COR", "cik": "0001140859", "name": "Cencora"},
    {"ticker": "CNC", "cik": "0001071739", "name": "Centene Corporation"},
    {"ticker": "CNP", "cik": "0001130310", "name": "CenterPoint Energy"},
    {"ticker": "CF", "cik": "0001324404", "name": "CF Industries"},
    {"ticker": "CRL", "cik": "0001100682", "name": "Charles River Laboratories"},
    {"ticker": "SCHW", "cik": "0000316709", "name": "Charles Schwab Corporation"},
    {"ticker": "CHTR", "cik": "0001091667", "name": "Charter Communications"},
    {"ticker": "CVX", "cik": "0000093410", "name": "Chevron Corporation"},
    {"ticker": "CMG", "cik": "0001058090", "name": "Chipotle Mexican Grill"},
    {"ticker": "CB", "cik": "0000896159", "name": "Chubb Limited"},
    {"ticker": "CHD", "cik": "0000313927", "name": "Church & Dwight"},
    {"ticker": "CI", "cik": "0001739940", "name": "Cigna"},
    {"ticker": "CINF", "cik": "0000020286", "name": "Cincinnati Financial"},
    {"ticker": "CTAS", "cik": "0000723254", "name": "Cintas"},
    {"ticker": "CSCO", "cik": "0000858877", "name": "Cisco"},
    {"ticker": "C", "cik": "0000831001", "name": "Citigroup"},
    {"ticker": "CFG", "cik": "0000759944", "name": "Citizens Financial Group"},
    {"ticker": "CLX", "cik": "0000021076", "name": "Clorox"},
    {"ticker": "CME", "cik": "0001156375", "name": "CME Group"},
    {"ticker": "CMS", "cik": "0000811156", "name": "CMS Energy"},
    {"ticker": "KO", "cik": "0000021344", "name": "Coca-Cola Company (The)"},
    {"ticker": "CTSH", "cik": "0001058290", "name": "Cognizant"},
    {"ticker": "COIN", "cik": "0001679788", "name": "Coinbase"},
    {"ticker": "CL", "cik": "0000021665", "name": "Colgate-Palmolive"},
    {"ticker": "CMCSA", "cik": "0001166691", "name": "Comcast"},
    {"ticker": "CAG", "cik": "0000023217", "name": "Conagra Brands"},
    {"ticker": "COP", "cik": "0001163165", "name": "ConocoPhillips"},
    {"ticker": "ED", "cik": "0001047862", "name": "Consolidated Edison"},
    {"ticker": "STZ", "cik": "0000016918", "name": "Constellation Brands"},
    {"ticker": "CEG", "cik": "0001868275", "name": "Constellation Energy"},
    {"ticker": "COO", "cik": "0000711404", "name": "Cooper Companies (The)"},
    {"ticker": "CPRT", "cik": "0000900075", "name": "Copart"},
    {"ticker": "GLW", "cik": "0000024741", "name": "Corning Inc."},
    {"ticker": "CPAY", "cik": "0001175454", "name": "Corpay"},
    {"ticker": "CTVA", "cik": "0001755672", "name": "Corteva"},
    {"ticker": "CSGP", "cik": "0001057352", "name": "CoStar Group"},
    {"ticker": "COST", "cik": "0000909832", "name": "Costco"},
    {"ticker": "CTRA", "cik": "0000858470", "name": "Coterra"},
    {"ticker": "CRWD", "cik": "0001535527", "name": "CrowdStrike"},
    {"ticker": "CCI", "cik": "0001051470", "name": "Crown Castle"},
    {"ticker": "CSX", "cik": "0000277948", "name": "CSX Corporation"},
    {"ticker": "CMI", "cik": "0000026172", "name": "Cummins"},
    {"ticker": "CVS", "cik": "0000064803", "name": "CVS Health"},
    {"ticker": "DHR", "cik": "0000313616", "name": "Danaher Corporation"},
    {"ticker": "DRI", "cik": "0000940944", "name": "Darden Restaurants"},
    {"ticker": "DDOG", "cik": "0001561550", "name": "Datadog"},
    {"ticker": "DVA", "cik": "0000927066", "name": "DaVita"},
    {"ticker": "DAY", "cik": "0001725057", "name": "Dayforce"},
    {"ticker": "DECK", "cik": "0000910521", "name": "Deckers Brands"},
    {"ticker": "DE", "cik": "0000315189", "name": "Deere & Company"},
    {"ticker": "DELL", "cik": "0001571996", "name": "Dell Technologies"},
    {"ticker": "DAL", "cik": "0000027904", "name": "Delta Air Lines"},
    {"ticker": "DVN", "cik": "0001090012", "name": "Devon Energy"},
    {"ticker": "DXCM", "cik": "0001093557", "name": "Dexcom"},
    {"ticker": "FANG", "cik": "0001539838", "name": "Diamondback Energy"},
    {"ticker": "DLR", "cik": "0001297996", "name": "Digital Realty"},
    {"ticker": "DG", "cik": "0000029534", "name": "Dollar General"},
    {"ticker": "DLTR", "cik": "0000935703", "name": "Dollar Tree"},
    {"ticker": "D", "cik": "0000715957", "name": "Dominion Energy"},
    {"ticker": "DPZ", "cik": "0001286681", "name": "Domino's"},
    {"ticker": "DASH", "cik": "0001792789", "name": "DoorDash"},
    {"ticker": "DOV", "cik": "0000029905", "name": "Dover Corporation"},
    {"ticker": "DOW", "cik": "0001751788", "name": "Dow Inc."},
    {"ticker": "DHI", "cik": "0000882184", "name": "D. R. Horton"},
    {"ticker": "DTE", "cik": "0000936340", "name": "DTE Energy"},
    {"ticker": "DUK", "cik": "0001326160", "name": "Duke Energy"},
    {"ticker": "DD", "cik": "0001666700", "name": "DuPont"},
    {"ticker": "EMN", "cik": "0000915389", "name": "Eastman Chemical Company"},
    {"ticker": "ETN", "cik": "0001551182", "name": "Eaton Corporation"},
    {"ticker": "EBAY", "cik": "0001065088", "name": "eBay Inc."},
    {"ticker": "ECL", "cik": "0000031462", "name": "Ecolab"},
    {"ticker": "EIX", "cik": "0000827052", "name": "Edison International"},
    {"ticker": "EW", "cik": "0001099800", "name": "Edwards Lifesciences"},
    {"ticker": "EA", "cik": "0000712515", "name": "Electronic Arts"},
    {"ticker": "ELV", "cik": "0001156039", "name": "Elevance Health"},
    {"ticker": "EMR", "cik": "0000032604", "name": "Emerson Electric"},
    {"ticker": "ENPH", "cik": "0001463101", "name": "Enphase Energy"},
    {"ticker": "ETR", "cik": "0000065984", "name": "Entergy"},
    {"ticker": "EOG", "cik": "0000821189", "name": "EOG Resources"},
    {"ticker": "EPAM", "cik": "0001352010", "name": "EPAM Systems"},
    {"ticker": "EQT", "cik": "0000033213", "name": "EQT Corporation"},
    {"ticker": "EFX", "cik": "0000033185", "name": "Equifax"},
    {"ticker": "EQIX", "cik": "0001101239", "name": "Equinix"},
    {"ticker": "EQR", "cik": "0000906107", "name": "Equity Residential"},
    {"ticker": "ERIE", "cik": "0000922621", "name": "Erie Indemnity"},
    {"ticker": "ESS", "cik": "0000920522", "name": "Essex Property Trust"},
    {"ticker": "EL", "cik": "0001001250", "name": "Estée Lauder Companies (The)"},
    {"ticker": "EG", "cik": "0001095073", "name": "Everest Group"},
    {"ticker": "EVRG", "cik": "0001711269", "name": "Evergy"},
    {"ticker": "ES", "cik": "0000072741", "name": "Eversource Energy"},
    {"ticker": "EXC", "cik": "0001109357", "name": "Exelon"},
    {"ticker": "EXE", "cik": "0000895126", "name": "Expand Energy"},
    {"ticker": "EXPE", "cik": "0001324424", "name": "Expedia Group"},
    {"ticker": "EXPD", "cik": "0000746515", "name": "Expeditors International"},
    {"ticker": "EXR", "cik": "0001289490", "name": "Extra Space Storage"},
    {"ticker": "XOM", "cik": "0000034088", "name": "ExxonMobil"},
    {"ticker": "FFIV", "cik": "0001048695", "name": "F5, Inc."},
    {"ticker": "FDS", "cik": "0001013237", "name": "FactSet"},
    {"ticker": "FICO", "cik": "0000814547", "name": "Fair Isaac"},
    {"ticker": "FAST", "cik": "0000815556", "name": "Fastenal"},
    {"ticker": "FRT", "cik": "0000034903", "name": "Federal Realty Investment Trust"},
    {"ticker": "FDX", "cik": "0001048911", "name": "FedEx"},
    {"ticker": "FIS", "cik": "0001136893", "name": "Fidelity National Information Services"},
    {"ticker": "FITB", "cik": "0000035527", "name": "Fifth Third Bancorp"},
    {"ticker": "FSLR", "cik": "0001274494", "name": "First Solar"},
    {"ticker": "FE", "cik": "0001031296", "name": "FirstEnergy"},
    {"ticker": "FI", "cik": "0000798354", "name": "Fiserv"},
    {"ticker": "F", "cik": "0000037996", "name": "Ford Motor Company"},
    {"ticker": "FTNT", "cik": "0001262039", "name": "Fortinet"},
    {"ticker": "FTV", "cik": "0001659166", "name": "Fortive"},
    {"ticker": "FOXA", "cik": "0001754301", "name": "Fox Corporation (Class A)"},
    {"ticker": "FOX", "cik": "0001754301", "name": "Fox Corporation (Class B)"},
    {"ticker": "BEN", "cik": "0000038777", "name": "Franklin Resources"},
    {"ticker": "FCX", "cik": "0000831259", "name": "Freeport-McMoRan"},
    {"ticker": "GRMN", "cik": "0001121788", "name": "Garmin"},
    {"ticker": "IT", "cik": "0000749251", "name": "Gartner"},
    {"ticker": "GE", "cik": "0000040545", "name": "GE Aerospace"},
    {"ticker": "GEHC", "cik": "0001932393", "name": "GE HealthCare"},
    {"ticker": "GEV", "cik": "0001996810", "name": "GE Vernova"},
    {"ticker": "GEN", "cik": "0000849399", "name": "Gen Digital"},
    {"ticker": "GNRC", "cik": "0001474735", "name": "Generac"},
    {"ticker": "GD", "cik": "0000040533", "name": "General Dynamics"},
    {"ticker": "GIS", "cik": "0000040704", "name": "General Mills"},
    {"ticker": "GM", "cik": "0001467858", "name": "General Motors"},
    {"ticker": "GPC", "cik": "0000040987", "name": "Genuine Parts Company"},
    {"ticker": "GILD", "cik": "0000882095", "name": "Gilead Sciences"},
    {"ticker": "GPN", "cik": "0001123360", "name": "Global Payments"},
    {"ticker": "GL", "cik": "0000320335", "name": "Globe Life"},
    {"ticker": "GDDY", "cik": "0001609711", "name": "GoDaddy"},
    {"ticker": "GS", "cik": "0000886982", "name": "Goldman Sachs"},
    {"ticker": "HAL", "cik": "0000045012", "name": "Halliburton"},
    {"ticker": "HIG", "cik": "0000874766", "name": "Hartford (The)"},
    {"ticker": "HAS", "cik": "0000046080", "name": "Hasbro"},
    {"ticker": "HCA", "cik": "0000860730", "name": "HCA Healthcare"},
    {"ticker": "DOC", "cik": "0000765880", "name": "Healthpeak Properties"},
    {"ticker": "HSIC", "cik": "0001000228", "name": "Henry Schein"},
    {"ticker": "HSY", "cik": "0000047111", "name": "Hershey Company (The)"},
    {"ticker": "HPE", "cik": "0001645590", "name": "Hewlett Packard Enterprise"},
    {"ticker": "HLT", "cik": "0001585689", "name": "Hilton Worldwide"},
    {"ticker": "HOLX", "cik": "0000859737", "name": "Hologic"},
    {"ticker": "HD", "cik": "0000354950", "name": "Home Depot (The)"},
    {"ticker": "HON", "cik": "0000773840", "name": "Honeywell"},
    {"ticker": "HRL", "cik": "0000048465", "name": "Hormel Foods"},
    {"ticker": "HST", "cik": "0001070750", "name": "Host Hotels & Resorts"},
    {"ticker": "HWM", "cik": "0000004281", "name": "Howmet Aerospace"},
    {"ticker": "HPQ", "cik": "0000047217", "name": "HP Inc."},
    {"ticker": "HUBB", "cik": "0000048898", "name": "Hubbell Incorporated"},
    {"ticker": "HUM", "cik": "0000049071", "name": "Humana"},
    {"ticker": "HBAN", "cik": "0000049196", "name": "Huntington Bancshares"},
    {"ticker": "HII", "cik": "0001501585", "name": "Huntington Ingalls Industries"},
    {"ticker": "IBM", "cik": "0000051143", "name": "IBM"},
    {"ticker": "IEX", "cik": "0000832101", "name": "IDEX Corporation"},
    {"ticker": "IDXX", "cik": "0000874716", "name": "Idexx Laboratories"},
    {"ticker": "ITW", "cik": "0000049826", "name": "Illinois Tool Works"},
    {"ticker": "INCY", "cik": "0000879169", "name": "Incyte"},
    {"ticker": "IR", "cik": "0001699150", "name": "Ingersoll Rand"},
    {"ticker": "PODD", "cik": "0001145197", "name": "Insulet Corporation"},
    {"ticker": "INTC", "cik": "0000050863", "name": "Intel"},
    {"ticker": "ICE", "cik": "0001571949", "name": "Intercontinental Exchange"},
    {"ticker": "IFF", "cik": "0000051253", "name": "International Flavors & Fragrances"},
    {"ticker": "IP", "cik": "0000051434", "name": "International Paper"},
    {"ticker": "IPG", "cik": "0000051644", "name": "Interpublic Group of Companies (The)"},
    {"ticker": "INTU", "cik": "0000896878", "name": "Intuit"},
    {"ticker": "ISRG", "cik": "0001035267", "name": "Intuitive Surgical"},
    {"ticker": "IVZ", "cik": "0000914208", "name": "Invesco"},
    {"ticker": "INVH", "cik": "0001687229", "name": "Invitation Homes"},
    {"ticker": "IQV", "cik": "0001478242", "name": "IQVIA"},
    {"ticker": "IRM", "cik": "0001020569", "name": "Iron Mountain"},
    {"ticker": "JBHT", "cik": "0000728535", "name": "J.B. Hunt"},
    {"ticker": "JBL", "cik": "0000898293", "name": "Jabil"},
    {"ticker": "JKHY", "cik": "0000779152", "name": "Jack Henry & Associates"},
    {"ticker": "J", "cik": "0000052988", "name": "Jacobs Solutions"},
    {"ticker": "JNJ", "cik": "0000200406", "name": "Johnson & Johnson"},
    {"ticker": "JCI", "cik": "0000833444", "name": "Johnson Controls"},
    {"ticker": "JPM", "cik": "0000019617", "name": "JPMorgan Chase"},
    {"ticker": "K", "cik": "0000055067", "name": "Kellanova"},
    {"ticker": "KVUE", "cik": "0001944048", "name": "Kenvue"},
    {"ticker": "KDP", "cik": "0001418135", "name": "Keurig Dr Pepper"},
    {"ticker": "KEY", "cik": "0000091576", "name": "KeyCorp"},
    {"ticker": "KEYS", "cik": "0001601046", "name": "Keysight Technologies"},
    {"ticker": "KMB", "cik": "0000055785", "name": "Kimberly-Clark"},
    {"ticker": "KIM", "cik": "0000879101", "name": "Kimco Realty"},
    {"ticker": "KMI", "cik": "0001506307", "name": "Kinder Morgan"},
    {"ticker": "KKR", "cik": "0001404912", "name": "KKR & Co."},
    {"ticker": "KLAC", "cik": "0000319201", "name": "KLA Corporation"},
    {"ticker": "KHC", "cik": "0001637459", "name": "Kraft Heinz"},
    {"ticker": "KR", "cik": "0000056873", "name": "Kroger"},
    {"ticker": "LHX", "cik": "0000202058", "name": "L3Harris"},
    {"ticker": "LH", "cik": "0000920148", "name": "Labcorp"},
    {"ticker": "LRCX", "cik": "0000707549", "name": "Lam Research"},
    {"ticker": "LW", "cik": "0001679273", "name": "Lamb Weston"},
    {"ticker": "LVS", "cik": "0001300514", "name": "Las Vegas Sands"},
    {"ticker": "LDOS", "cik": "0001336920", "name": "Leidos"},
    {"ticker": "LEN", "cik": "0000920760", "name": "Lennar"},
    {"ticker": "LII", "cik": "0001069202", "name": "Lennox International"},
    {"ticker": "LLY", "cik": "0000059478", "name": "Lilly (Eli)"},
    {"ticker": "LIN", "cik": "0001707925", "name": "Linde plc"},
    {"ticker": "LYV", "cik": "0001335258", "name": "Live Nation Entertainment"},
    {"ticker": "LKQ", "cik": "0001065696", "name": "LKQ Corporation"},
    {"ticker": "LMT", "cik": "0000936468", "name": "Lockheed Martin"},
    {"ticker": "L", "cik": "0000060086", "name": "Loews Corporation"},
    {"ticker": "LOW", "cik": "0000060667", "name": "Lowe's"},
    {"ticker": "LULU", "cik": "0001397187", "name": "Lululemon Athletica"},
    {"ticker": "LYB", "cik": "0001489393", "name": "LyondellBasell"},
    {"ticker": "MTB", "cik": "0000036270", "name": "M&T Bank"},
    {"ticker": "MPC", "cik": "0001510295", "name": "Marathon Petroleum"},
    {"ticker": "MKTX", "cik": "0001278021", "name": "MarketAxess"},
    {"ticker": "MAR", "cik": "0001048286", "name": "Marriott International"},
    {"ticker": "MMC", "cik": "0000062709", "name": "Marsh McLennan"},
    {"ticker": "MLM", "cik": "0000916076", "name": "Martin Marietta Materials"},
    {"ticker": "MAS", "cik": "0000062996", "name": "Masco"},
    {"ticker": "MA", "cik": "0001141391", "name": "Mastercard"},
    {"ticker": "MTCH", "cik": "0000891103", "name": "Match Group"},
    {"ticker": "MKC", "cik": "0000063754", "name": "McCormick & Company"},
    {"ticker": "MCD", "cik": "0000063908", "name": "McDonald's"},
    {"ticker": "MCK", "cik": "0000927653", "name": "McKesson Corporation"},
    {"ticker": "MDT", "cik": "0001613103", "name": "Medtronic"},
    {"ticker": "MRK", "cik": "0000310158", "name": "Merck & Co."},
    {"ticker": "META", "cik": "0001326801", "name": "Meta Platforms"},
    {"ticker": "MET", "cik": "0001099219", "name": "MetLife"},
    {"ticker": "MTD", "cik": "0001037646", "name": "Mettler Toledo"},
    {"ticker": "MGM", "cik": "0000789570", "name": "MGM Resorts"},
    {"ticker": "MCHP", "cik": "0000827054", "name": "Microchip Technology"},
    {"ticker": "MU", "cik": "0000723125", "name": "Micron Technology"},
    {"ticker": "MSFT", "cik": "0000789019", "name": "Microsoft"},
    {"ticker": "MAA", "cik": "0000912595", "name": "Mid-America Apartment Communities"},
    {"ticker": "MRNA", "cik": "0001682852", "name": "Moderna"},
    {"ticker": "MHK", "cik": "0000851968", "name": "Mohawk Industries"},
    {"ticker": "MOH", "cik": "0001179929", "name": "Molina Healthcare"},
    {"ticker": "TAP", "cik": "0000024545", "name": "Molson Coors Beverage Company"},
    {"ticker": "MDLZ", "cik": "0001103982", "name": "Mondelez International"},
    {"ticker": "MPWR", "cik": "0001280452", "name": "Monolithic Power Systems"},
    {"ticker": "MNST", "cik": "0000865752", "name": "Monster Beverage"},
    {"ticker": "MCO", "cik": "0001059556", "name": "Moody's Corporation"},
    {"ticker": "MS", "cik": "0000895421", "name": "Morgan Stanley"},
    {"ticker": "MOS", "cik": "0001285785", "name": "Mosaic Company (The)"},
    {"ticker": "MSI", "cik": "0000068505", "name": "Motorola Solutions"},
    {"ticker": "MSCI", "cik": "0001408198", "name": "MSCI Inc."},
    {"ticker": "NDAQ", "cik": "0001120193", "name": "Nasdaq, Inc."},
    {"ticker": "NTAP", "cik": "0001002047", "name": "NetApp"},
    {"ticker": "NFLX", "cik": "0001065280", "name": "Netflix"},
    {"ticker": "NEM", "cik": "0001164727", "name": "Newmont"},
    {"ticker": "NWSA", "cik": "0001564708", "name": "News Corp (Class A)"},
    {"ticker": "NWS", "cik": "0001564708", "name": "News Corp (Class B)"},
    {"ticker": "NEE", "cik": "0000753308", "name": "NextEra Energy"},
    {"ticker": "NKE", "cik": "0000320187", "name": "Nike, Inc."},
    {"ticker": "NI", "cik": "0001111711", "name": "NiSource"},
    {"ticker": "NDSN", "cik": "0000072331", "name": "Nordson Corporation"},
    {"ticker": "NSC", "cik": "0000702165", "name": "Norfolk Southern"},
    {"ticker": "NTRS", "cik": "0000073124", "name": "Northern Trust"},
    {"ticker": "NOC", "cik": "0001133421", "name": "Northrop Grumman"},
    {"ticker": "NCLH", "cik": "0001513761", "name": "Norwegian Cruise Line Holdings"},
    {"ticker": "NRG", "cik": "0001013871", "name": "NRG Energy"},
    {"ticker": "NUE", "cik": "0000073309", "name": "Nucor"},
    {"ticker": "NVDA", "cik": "0001045810", "name": "Nvidia"},
    {"ticker": "NVR", "cik": "0000906163", "name": "NVR, Inc."},
    {"ticker": "NXPI", "cik": "0001413447", "name": "NXP Semiconductors"},
    {"ticker": "ORLY", "cik": "0000898173", "name": "O'Reilly Automotive"},
    {"ticker": "OXY", "cik": "0000797468", "name": "Occidental Petroleum"},
    {"ticker": "ODFL", "cik": "0000878927", "name": "Old Dominion"},
    {"ticker": "OMC", "cik": "0000029989", "name": "Omnicom Group"},
    {"ticker": "ON", "cik": "0001097864", "name": "ON Semiconductor"},
    {"ticker": "OKE", "cik": "0001039684", "name": "Oneok"},
    {"ticker": "ORCL", "cik": "0001341439", "name": "Oracle Corporation"},
    {"ticker": "OTIS", "cik": "0001781335", "name": "Otis Worldwide"},
    {"ticker": "PCAR", "cik": "0000075362", "name": "Paccar"},
    {"ticker": "PKG", "cik": "0000075677", "name": "Packaging Corporation of America"},
    {"ticker": "PLTR", "cik": "0001321655", "name": "Palantir Technologies"},
    {"ticker": "PANW", "cik": "0001327567", "name": "Palo Alto Networks"},
    {"ticker": "PSKY", "cik": "0002041610", "name": "Paramount Skydance Corporation"},
    {"ticker": "PH", "cik": "0000076334", "name": "Parker Hannifin"},
    {"ticker": "PAYX", "cik": "0000723531", "name": "Paychex"},
    {"ticker": "PAYC", "cik": "0001590955", "name": "Paycom"},
    {"ticker": "PYPL", "cik": "0001633917", "name": "PayPal"},
    {"ticker": "PNR", "cik": "0000077360", "name": "Pentair"},
    {"ticker": "PEP", "cik": "0000077476", "name": "PepsiCo"},
    {"ticker": "PFE", "cik": "0000078003", "name": "Pfizer"},
    {"ticker": "PCG", "cik": "0001004980", "name": "PG&E Corporation"},
    {"ticker": "PM", "cik": "0001413329", "name": "Philip Morris International"},
    {"ticker": "PSX", "cik": "0001534701", "name": "Phillips 66"},
    {"ticker": "PNW", "cik": "0000764622", "name": "Pinnacle West Capital"},
    {"ticker": "PNC", "cik": "0000713676", "name": "PNC Financial Services"},
    {"ticker": "POOL", "cik": "0000945841", "name": "Pool Corporation"},
    {"ticker": "PPG", "cik": "0000079879", "name": "PPG Industries"},
    {"ticker": "PPL", "cik": "0000922224", "name": "PPL Corporation"},
    {"ticker": "PFG", "cik": "0001126328", "name": "Principal Financial Group"},
    {"ticker": "PG", "cik": "0000080424", "name": "Procter & Gamble"},
    {"ticker": "PGR", "cik": "0000080661", "name": "Progressive Corporation"},
    {"ticker": "PLD", "cik": "0001045609", "name": "Prologis"},
    {"ticker": "PRU", "cik": "0001137774", "name": "Prudential Financial"},
    {"ticker": "PEG", "cik": "0000788784", "name": "Public Service Enterprise Group"},
    {"ticker": "PTC", "cik": "0000857005", "name": "PTC Inc."},
    {"ticker": "PSA", "cik": "0001393311", "name": "Public Storage"},
    {"ticker": "PHM", "cik": "0000822416", "name": "PulteGroup"},
    {"ticker": "PWR", "cik": "0001050915", "name": "Quanta Services"},
    {"ticker": "QCOM", "cik": "0000804328", "name": "Qualcomm"},
    {"ticker": "DGX", "cik": "0001022079", "name": "Quest Diagnostics"},
    {"ticker": "RL", "cik": "0001037038", "name": "Ralph Lauren Corporation"},
    {"ticker": "RJF", "cik": "0000720005", "name": "Raymond James Financial"},
    {"ticker": "RTX", "cik": "0000101829", "name": "RTX Corporation"},
    {"ticker": "O", "cik": "0000726728", "name": "Realty Income"},
    {"ticker": "REG", "cik": "0000910606", "name": "Regency Centers"},
    {"ticker": "REGN", "cik": "0000872589", "name": "Regeneron Pharmaceuticals"},
    {"ticker": "RF", "cik": "0001281761", "name": "Regions Financial Corporation"},
    {"ticker": "RSG", "cik": "0001060391", "name": "Republic Services"},
    {"ticker": "RMD", "cik": "0000943819", "name": "ResMed"},
    {"ticker": "RVTY", "cik": "0000031791", "name": "Revvity"},
    {"ticker": "ROK", "cik": "0001024478", "name": "Rockwell Automation"},
    {"ticker": "ROL", "cik": "0000084839", "name": "Rollins, Inc."},
    {"ticker": "ROP", "cik": "0000882835", "name": "Roper Technologies"},
    {"ticker": "ROST", "cik": "0000745732", "name": "Ross Stores"},
    {"ticker": "RCL", "cik": "0000884887", "name": "Royal Caribbean Group"},
    {"ticker": "SPGI", "cik": "0000064040", "name": "S&P Global"},
    {"ticker": "CRM", "cik": "0001108524", "name": "Salesforce"},
    {"ticker": "SBAC", "cik": "0001034054", "name": "SBA Communications"},
    {"ticker": "SLB", "cik": "0000087347", "name": "Schlumberger"},
    {"ticker": "STX", "cik": "0001137789", "name": "Seagate Technology"},
    {"ticker": "SRE", "cik": "0001032208", "name": "Sempra"},
    {"ticker": "NOW", "cik": "0001373715", "name": "ServiceNow"},
    {"ticker": "SHW", "cik": "0000089800", "name": "Sherwin-Williams"},
    {"ticker": "SPG", "cik": "0001063761", "name": "Simon Property Group"},
    {"ticker": "SWKS", "cik": "0000004127", "name": "Skyworks Solutions"},
    {"ticker": "SJM", "cik": "0000091419", "name": "J.M. Smucker Company (The)"},
    {"ticker": "SW", "cik": "0002005951", "name": "Smurfit Westrock"},
    {"ticker": "SNA", "cik": "0000091440", "name": "Snap-on"},
    {"ticker": "SOLV", "cik": "0001964738", "name": "Solventum"},
    {"ticker": "SO", "cik": "0000092122", "name": "Southern Company"},
    {"ticker": "LUV", "cik": "0000092380", "name": "Southwest Airlines"},
    {"ticker": "SWK", "cik": "0000093556", "name": "Stanley Black & Decker"},
    {"ticker": "SBUX", "cik": "0000829224", "name": "Starbucks"},
    {"ticker": "STT", "cik": "0000093751", "name": "State Street Corporation"},
    {"ticker": "STLD", "cik": "0001022671", "name": "Steel Dynamics"},
    {"ticker": "STE", "cik": "0001757898", "name": "Steris"},
    {"ticker": "SYK", "cik": "0000310764", "name": "Stryker Corporation"},
    {"ticker": "SMCI", "cik": "0001375365", "name": "Supermicro"},
    {"ticker": "SYF", "cik": "0001601712", "name": "Synchrony Financial"},
    {"ticker": "SNPS", "cik": "0000883241", "name": "Synopsys"},
    {"ticker": "SYY", "cik": "0000096021", "name": "Sysco"},
    {"ticker": "TMUS", "cik": "0001283699", "name": "T-Mobile US"},
    {"ticker": "TROW", "cik": "0001113169", "name": "T. Rowe Price"},
    {"ticker": "TTWO", "cik": "0000946581", "name": "Take-Two Interactive"},
    {"ticker": "TPR", "cik": "0001116132", "name": "Tapestry, Inc."},
    {"ticker": "TRGP", "cik": "0001389170", "name": "Targa Resources"},
    {"ticker": "TGT", "cik": "0000027419", "name": "Target Corporation"},
    {"ticker": "TEL", "cik": "0001385157", "name": "TE Connectivity"},
    {"ticker": "TDY", "cik": "0001094285", "name": "Teledyne Technologies"},
    {"ticker": "TER", "cik": "0000097210", "name": "Teradyne"},
    {"ticker": "TSLA", "cik": "0001318605", "name": "Tesla, Inc."},
    {"ticker": "TXN", "cik": "0000097476", "name": "Texas Instruments"},
    {"ticker": "TPL", "cik": "0001811074", "name": "Texas Pacific Land Corporation"},
    {"ticker": "TXT", "cik": "0000217346", "name": "Textron"},
    {"ticker": "TMO", "cik": "0000097745", "name": "Thermo Fisher Scientific"},
    {"ticker": "TJX", "cik": "0000109198", "name": "TJX Companies"},
    {"ticker": "TKO", "cik": "0001973266", "name": "TKO Group Holdings"},
    {"ticker": "TTD", "cik": "0001671933", "name": "Trade Desk (The)"},
    {"ticker": "TSCO", "cik": "0000916365", "name": "Tractor Supply"},
    {"ticker": "TT", "cik": "0001466258", "name": "Trane Technologies"},
    {"ticker": "TDG", "cik": "0001260221", "name": "TransDigm Group"},
    {"ticker": "TRV", "cik": "0000086312", "name": "Travelers Companies (The)"},
    {"ticker": "TRMB", "cik": "0000864749", "name": "Trimble Inc."},
    {"ticker": "TFC", "cik": "0000092230", "name": "Truist Financial"},
    {"ticker": "TYL", "cik": "0000860731", "name": "Tyler Technologies"},
    {"ticker": "TSN", "cik": "0000100493", "name": "Tyson Foods"},
    {"ticker": "USB", "cik": "0000036104", "name": "U.S. Bancorp"},
    {"ticker": "UBER", "cik": "0001543151", "name": "Uber"},
    {"ticker": "UDR", "cik": "0000074208", "name": "UDR, Inc."},
    {"ticker": "ULTA", "cik": "0001403568", "name": "Ulta Beauty"},
    {"ticker": "UNP", "cik": "0000100885", "name": "Union Pacific Corporation"},
    {"ticker": "UAL", "cik": "0000100517", "name": "United Airlines Holdings"},
    {"ticker": "UPS", "cik": "0001090727", "name": "United Parcel Service"},
    {"ticker": "URI", "cik": "0001067701", "name": "United Rentals"},
    {"ticker": "UNH", "cik": "0000731766", "name": "UnitedHealth Group"},
    {"ticker": "UHS", "cik": "0000352915", "name": "Universal Health Services"},
    {"ticker": "VLO", "cik": "0001035002", "name": "Valero Energy"},
    {"ticker": "VTR", "cik": "0000740260", "name": "Ventas"},
    {"ticker": "VLTO", "cik": "0001967680", "name": "Veralto"},
    {"ticker": "VRSN", "cik": "0001014473", "name": "Verisign"},
    {"ticker": "VRSK", "cik": "0001442145", "name": "Verisk Analytics"},
    {"ticker": "VZ", "cik": "0000732712", "name": "Verizon"},
    {"ticker": "VRTX", "cik": "0000875320", "name": "Vertex Pharmaceuticals"},
    {"ticker": "VTRS", "cik": "0001792044", "name": "Viatris"},
    {"ticker": "VICI", "cik": "0001705696", "name": "Vici Properties"},
    {"ticker": "V", "cik": "0001403161", "name": "Visa Inc."},
    {"ticker": "VST", "cik": "0001692819", "name": "Vistra Corp."},
    {"ticker": "VMC", "cik": "0001396009", "name": "Vulcan Materials Company"},
    {"ticker": "WRB", "cik": "0000011544", "name": "W. R. Berkley Corporation"},
    {"ticker": "GWW", "cik": "0000277135", "name": "W. W. Grainger"},
    {"ticker": "WAB", "cik": "0000943452", "name": "Wabtec"},
    {"ticker": "WBA", "cik": "0001618921", "name": "Walgreens Boots Alliance"},
    {"ticker": "WMT", "cik": "0000104169", "name": "Walmart"},
    {"ticker": "DIS", "cik": "0001744489", "name": "Walt Disney Company (The)"},
    {"ticker": "WBD", "cik": "0001437107", "name": "Warner Bros. Discovery"},
    {"ticker": "WM", "cik": "0000823768", "name": "Waste Management"},
    {"ticker": "WAT", "cik": "0001000697", "name": "Waters Corporation"},
    {"ticker": "WEC", "cik": "0000783325", "name": "WEC Energy Group"},
    {"ticker": "WFC", "cik": "0000072971", "name": "Wells Fargo"},
    {"ticker": "WELL", "cik": "0000766704", "name": "Welltower"},
    {"ticker": "WST", "cik": "0000105770", "name": "West Pharmaceutical Services"},
    {"ticker": "WDC", "cik": "0000106040", "name": "Western Digital"},
    {"ticker": "WY", "cik": "0000106535", "name": "Weyerhaeuser"},
    {"ticker": "WSM", "cik": "0000719955", "name": "Williams-Sonoma, Inc."},
    {"ticker": "WMB", "cik": "0000107263", "name": "Williams Companies"},
    {"ticker": "WTW", "cik": "0001140536", "name": "Willis Towers Watson"},
    {"ticker": "WDAY", "cik": "0001327811", "name": "Workday, Inc."},
    {"ticker": "WYNN", "cik": "0001174922", "name": "Wynn Resorts"},
    {"ticker": "XEL", "cik": "0000072903", "name": "Xcel Energy"},
    {"ticker": "XYL", "cik": "0001524472", "name": "Xylem Inc."},
    {"ticker": "YUM", "cik": "0001041061", "name": "Yum! Brands"},
    {"ticker": "ZBRA", "cik": "0000877212", "name": "Zebra Technologies"},
    {"ticker": "ZBH", "cik": "0001136869", "name": "Zimmer Biomet"},
    {"ticker": "ZTS", "cik": "0001555280", "name": "Zoetis"}
]

# Total companies: 503

# Total companies: 503

companies_df = pd.DataFrame(sp500_companies)
print(f"Loaded {len(companies_df)} companies")
companies_df.head()

Loaded 503 companies


,ticker,cik,name
0,MMM,0000066740,3M
1,AOS,0000091142,A. O. Smith
2,ABT,0000001800,Abbott Laboratories
3,ABBV,0001551152,AbbVie
4,ACN,0001467373,Accenture


## 3. Download 10-K Filings

In [17]:
# Initialize downloader
downloader = SECDownloader(user_agent=USER_AGENT, data_dir=RAW_DATA_DIR)

# Download filings
download_log = []

for idx, company in tqdm(companies_df.iterrows(), total=len(companies_df), desc="Downloading 10-Ks"):
    ticker = company['ticker']
    cik = company['cik']
    
    print(f"\nProcessing {ticker} ({company['name']})...")
    
    try:
        # Download 10-K filings
        filings = downloader.download_10k(
            cik=cik,
            ticker=ticker,
            start_year=START_YEAR,
            end_year=END_YEAR,
            prefer_html=PREFER_HTML,
            max_filings=TARGET_YEARS
        )
        
        for filing in filings:
            download_log.append({
                'ticker': ticker,
                'cik': cik,
                'fiscal_year': filing['fiscal_year'],
                'file_format': filing['format'],
                'file_path': filing['path'],
                'download_date': datetime.now().isoformat(),
                'status': 'success'
            })
        
        # SEC rate limiting: 10 requests per second
        time.sleep(0.05)
        
    except Exception as e:
        print(f"Error downloading {ticker}: {str(e)}")
        download_log.append({
            'ticker': ticker,
            'cik': cik,
            'fiscal_year': None,
            'file_format': None,
            'file_path': None,
            'download_date': datetime.now().isoformat(),
            'status': f'error: {str(e)}'
        })


Processing MMM (3M)...
  Downloading: https://www.sec.gov/Archives/edgar/data/66740/000006674024000016/mmm-20231231.htm
  ✓ Saved 4286095 bytes to MMM_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/66740/000006674023000014/mmm-20221231.htm
  ✓ Saved 6551646 bytes to MMM_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/66740/000006674022000010/mmm-20211231.htm


  ✓ Saved 6745566 bytes to MMM_2022_10K.html

Processing AOS (A. O. Smith)...
  Downloading: https://www.sec.gov/Archives/edgar/data/91142/000009114224000041/aos-20231231.htm
  ✓ Saved 1914385 bytes to AOS_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/91142/000009114223000025/aos-20221231.htm
  ✓ Saved 2511727 bytes to AOS_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/91142/000009114222000028/aos-20211231.htm
  ✓ Saved 2471981 bytes to AOS_2022_10K.html



Processing ABT (Abbott Laboratories)...
  Downloading: https://www.sec.gov/Archives/edgar/data/1800/000162828024005348/abt-20231231.htm
  ✓ Saved 2675005 bytes to ABT_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1800/000162828023004026/abt-20221231.htm
  ✓ Saved 3412193 bytes to ABT_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1800/000110465922025141/abt-20211231x10k.htm
  ✓ Saved 4169789 bytes to ABT_2022_10K.html



Processing ABBV (AbbVie)...
  Downloading: https://www.sec.gov/Archives/edgar/data/1551152/000155115224000011/abbv-20231231.htm
  ✓ Saved 3701766 bytes to ABBV_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1551152/000155115223000011/abbv-20221231.htm
  ✓ Saved 4840548 bytes to ABBV_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1551152/000155115222000007/abbv-20211231.htm
  ✓ Saved 4993086 bytes to ABBV_2022_10K.html



Processing ACN (Accenture)...
  Downloading: https://www.sec.gov/Archives/edgar/data/1467373/000146737324000278/acn-20240831.htm
  ✓ Saved 2789648 bytes to ACN_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1467373/000146737323000324/acn-20230831.htm
  ✓ Saved 2767778 bytes to ACN_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1467373/000146737322000295/acn-20220831.htm
  ✓ Saved 3381988 bytes to ACN_2022_10K.html


...

  ✓ Saved 6752221 bytes to XEL_2022_10K.html

Processing XYL (Xylem Inc.)...
  Downloading: https://www.sec.gov/Archives/edgar/data/1524472/000152447224000006/xyl-20231231.htm
  ✓ Saved 3453993 bytes to XYL_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1524472/000152447223000009/xyl-20221231.htm
  ✓ Saved 3930827 bytes to XYL_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1524472/000152447222000009/xyl-20211231.htm


  ✓ Saved 3834462 bytes to XYL_2022_10K.html

Processing YUM (Yum! Brands)...
  Downloading: https://www.sec.gov/Archives/edgar/data/1041061/000104106124000011/yum-20231231.htm
  ✓ Saved 3095561 bytes to YUM_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1041061/000104106123000009/yum-20221231.htm
  ✓ Saved 4061824 bytes to YUM_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1041061/000104106122000009/yum-20211231.htm


  ✓ Saved 4089611 bytes to YUM_2022_10K.html

Processing ZBRA (Zebra Technologies)...
  Downloading: https://www.sec.gov/Archives/edgar/data/877212/000087721224000029/zbra-20231231.htm
  ✓ Saved 2271951 bytes to ZBRA_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/877212/000087721223000025/zbra-20221231.htm
  ✓ Saved 2928899 bytes to ZBRA_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/877212/000087721222000026/zbra-20211231.htm


  ✓ Saved 2958974 bytes to ZBRA_2022_10K.html

Processing ZBH (Zimmer Biomet)...
  Downloading: https://www.sec.gov/Archives/edgar/data/1136869/000095017024019353/zbh-20231231.htm
  ✓ Saved 5117893 bytes to ZBH_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1136869/000095017023004264/zbh-20221231.htm
  ✓ Saved 7665395 bytes to ZBH_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1136869/000156459022007160/zbh-10k_20211231.htm


  ✓ Saved 7300638 bytes to ZBH_2022_10K.html

Processing ZTS (Zoetis)...
  Downloading: https://www.sec.gov/Archives/edgar/data/1555280/000155528024000143/zts-20231231.htm
  ✓ Saved 3288013 bytes to ZTS_2024_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1555280/000155528023000074/zts-20221231.htm
  ✓ Saved 4067415 bytes to ZTS_2023_10K.html
  Downloading: https://www.sec.gov/Archives/edgar/data/1555280/000155528022000078/zts-20211231.htm
  ✓ Saved 4141808 bytes to ZTS_2022_10K.html


## 4. Download XBRL Data

In [18]:
# Download XBRL financial statement data sets
xbrl_log = []

for idx, company in tqdm(companies_df.iterrows(), total=len(companies_df), desc="Downloading XBRL"):
    ticker = company['ticker']
    cik = company['cik']
    
    try:
        xbrl_files = downloader.download_xbrl(
            cik=cik,
            ticker=ticker,
            start_year=START_YEAR,
            end_year=END_YEAR
        )
        
        for xbrl in xbrl_files:
            xbrl_log.append({
                'ticker': ticker,
                'cik': cik,
                'fiscal_year': xbrl['fiscal_year'],
                'file_path': xbrl['path'],
                'status': 'success'
            })
        
        time.sleep(0.05)
        
    except Exception as e:
        print(f"Error downloading XBRL for {ticker}: {str(e)}")
        xbrl_log.append({
            'ticker': ticker,
            'cik': cik,
            'status': f'error: {str(e)}'
        })

XBRL download for MMM - Implementation needed
XBRL download for AOS - Implementation needed
XBRL download for ABT - Implementation needed
XBRL download for ABBV - Implementation needed


XBRL download for ACN - Implementation needed
XBRL download for ADBE - Implementation needed
XBRL download for AMD - Implementation needed
XBRL download for AES - Implementation needed


XBRL download for AFL - Implementation needed
XBRL download for A - Implementation needed
XBRL download for APD - Implementation needed
XBRL download for ABNB - Implementation needed


XBRL download for AKAM - Implementation needed
XBRL download for ALB - Implementation needed
XBRL download for ARE - Implementation needed
XBRL download for ALGN - Implementation needed


XBRL download for ALLE - Implementation needed
XBRL download for LNT - Implementation needed
XBRL download for ALL - Implementation needed
XBRL download for GOOGL - Implementation needed


...

XBRL download for WM - Implementation needed
XBRL download for WAT - Implementation needed
XBRL download for WEC - Implementation needed
XBRL download for WFC - Implementation needed


XBRL download for WELL - Implementation needed
XBRL download for WST - Implementation needed
XBRL download for WDC - Implementation needed
XBRL download for WY - Implementation needed


XBRL download for WSM - Implementation needed
XBRL download for WMB - Implementation needed
XBRL download for WTW - Implementation needed
XBRL download for WDAY - Implementation needed


XBRL download for WYNN - Implementation needed
XBRL download for XEL - Implementation needed
XBRL download for XYL - Implementation needed
XBRL download for YUM - Implementation needed


XBRL download for ZBRA - Implementation needed
XBRL download for ZBH - Implementation needed
XBRL download for ZTS - Implementation needed


## 5. Summary and Validation

In [19]:
# Create download summary
download_df = pd.DataFrame(download_log)
xbrl_df = pd.DataFrame(xbrl_log)

# Save logs
download_df.to_csv(RAW_DATA_DIR / 'download_log.csv', index=False)
if len(xbrl_df) > 0:
    xbrl_df.to_csv(RAW_DATA_DIR / 'xbrl_log.csv', index=False)

print("\n=== Download Summary ===")
print(f"Total filings downloaded: {len(download_df[download_df['status'] == 'success'])}")

# Handle empty XBRL data
if len(xbrl_df) > 0 and 'status' in xbrl_df.columns:
    print(f"Total XBRL files downloaded: {len(xbrl_df[xbrl_df['status'] == 'success'])}")
else:
    print("Total XBRL files downloaded: 0 (XBRL download not implemented)")

print(f"\nFilings by format:")
print(download_df[download_df['status'] == 'success']['file_format'].value_counts())
print(f"\nFilings by year:")
print(download_df[download_df['status'] == 'success']['fiscal_year'].value_counts().sort_index())

# Display first few rows
download_df.head(10)



=== Download Summary ===
Total filings downloaded: 1481
Total XBRL files downloaded: 0 (XBRL download not implemented)

Filings by format:
file_format
html    1481
Name: count, dtype: int64

Filings by year:
fiscal_year
2021      2
2022    491
2023    492
2024    496
Name: count, dtype: int64


,ticker,cik,fiscal_year,file_format,file_path,download_date,status
0,MMM,0000066740,2024,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:27.690989,success
1,MMM,0000066740,2023,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:27.690989,success
2,MMM,0000066740,2022,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:27.690989,success
3,AOS,0000091142,2024,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:28.987291,success
4,AOS,0000091142,2023,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:28.987291,success
5,AOS,0000091142,2022,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:28.987291,success
6,ABT,0000001800,2024,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:30.288443,success
7,ABT,0000001800,2023,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:30.288443,success
8,ABT,0000001800,2022,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:30.288443,success
9,ABBV,0001551152,2024,html,c:\Users\anand\Desktop\SEM 3\CS 582\Proj\data\...,2025-12-03T11:16:31.777414,success


## 6. Data Quality Checks

In [20]:
# Check for missing filings
successful_downloads = download_df[download_df['status'] == 'success']
failed_downloads = download_df[download_df['status'] != 'success']

print(f"Success rate: {len(successful_downloads) / len(download_df) * 100:.2f}%")

if len(failed_downloads) > 0:
    print("\nFailed downloads:")
    print(failed_downloads[['ticker', 'fiscal_year', 'status']])

# Check file sizes
for _, row in successful_downloads.iterrows():
    if row['file_path'] and os.path.exists(row['file_path']):
        size_mb = os.path.getsize(row['file_path']) / (1024 * 1024)
        if size_mb < 0.1:
            print(f"Warning: Small file detected - {row['ticker']} {row['fiscal_year']}: {size_mb:.2f} MB")

Success rate: 100.00%


## Next Steps

Proceed to **02_document_parsing.ipynb** to parse the downloaded filings into structured components (text, tables, figures).